In [8]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
#from scipy import interp, arange, exp
#from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
import shutil

from scipy.interpolate import make_interp_spline
from scipy.stats import ks_2samp
from scipy.spatial import ConvexHull

In [9]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import plotlib

In [10]:
with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/output_repar_03-01/'               # Harmon server
name_project

'project_repartition_v4.0/output_repar_v9.5_04--10-16Hz/'

In [11]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.5_04--10-16Hz/


In [12]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [13]:
rdir_partition = 'partition/'
dir_partition = dir_project + rdir_partition
info_basic['rdir_partition'] = rdir_partition
if os.path.exists(dir_partition):
    shutil.rmtree(dir_partition)
os.makedirs(dir_partition)

In [20]:
file_partition_num = dir_project + 'partition_nums.txt'
partition_nums = np.loadtxt(file_partition_num, dtype=int)

In [21]:
index_partition = []
key_subworks = []
nums_new = []
dir_partition_old = dir_project_workspace + 'project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/partition/'
old_partitions = os.listdir(dir_partition_old)
for file in old_partitions:
    if file.endswith('.txt'):
        this_num = int(file.split('--')[0])
        if this_num in partition_nums:
            index_partition.append(old_partitions.index(file))
            nums_new.append(this_num)
            key_subworks.append(file.split('.')[0])
            shutil.copy(dir_partition_old+file, dir_partition+file)
for num in partition_nums:
    if num not in nums_new:
        print('Partition num: ', num, ' is missing!')

In [22]:
info_basic['key_subworks']  = key_subworks
info_basic['stalistname_all'] = 'stations_info_all.xlsx'

In [23]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)
np.save(dir_project + 'Basic_info.npy', info_basic_bi)